##### Copyright 2020 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Transforming circuits

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/transform"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/transform.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/transform.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/transform.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

## Circuit optimizers

Cirq comes with the concept of an optimizer.  Optimizers will pass over a circuit and perform tasks that will modify the circuit in place.  These can be used to transform a circuit in specific ways, such as combining single-qubit gates, commuting Z gates through the circuit, or readying the circuit for certain hardware or gate set configurations.

Optimizers will have a function `optimize_circuit()` that can be used to perform this optimization.  Here is a simple example that removes empty moments:

In [ ]:
import cirq
c=cirq.Circuit()
c.append(cirq.Moment([]))
c.append(cirq.Moment([cirq.X(cirq.GridQubit(1,1))]))
c.append(cirq.Moment([]))
print(f'Before optimization, Circuit has {len(c)} moments')

cirq.DropEmptyMoments().optimize_circuit(circuit=c)
print(f'After optimization, Circuit has {len(c)} moments')

Optimizers that come with cirq can be found in the `cirq.optimizers` package.

A few notable examples are:

*   **ConvertToCzAndSingleGates**: Attempts to convert a circuit into CZ gates and single qubit gates.  This uses gate's unitary and decompose methods to transform them into CZ + single qubit gates.
*   **DropEmptyMoments** / **DropNegligible**:  Removes moments that are empty or have very small effects, respectively.
*   **EjectPhasedPaulis**: Pushes X, Y, and PhasedX gates towards the end of the circuit, potentially absorbing Z gates and modifying gates along the way.
*   **EjectZ**:  Pushes Z gates towards the end of the circuit, potentially adjusting phases of gates that they pass through.
*   **ExpandComposite**:  Uses `cirq.decompose` to expand composite gates.
*   **MergeInteractions**:  Combines series of adjacent one and two-qubit gates acting on a pair of qubits.
*   **MergeSingleQubitGates**:  Combines series of adjacent unitary 1-qubit operations
*   **SynchronizeTerminalMeasurements**:  Moves all measurements in a circuit to the final moment if possible.


### Create your own optimizers

You can create your own optimizers to transform and modify circuits to fit hardware, gate sets, or other requirements.  Optimizers can also be used to generate noise.  See [noise](noise.ipynb) for details.

You can do this by implementing the function `optimize_circuit`.

If your optimizer is a local optimizer and depends primarily on operator being examined, you can alternatively inherit `cirq.PointOptimizer` and implement the function `optimization_at(self, circuit, index, op)` that optimizes a single operation.

Below is an example of implementing a simple `PointOptimizer` that removes measurements.

In [ ]:
class RemoveMeasurements(cirq.PointOptimizer):
    def optimization_at(self, circuit: cirq.Circuit, index: int, op: cirq.Operation):
      if isinstance(op.gate, cirq.MeasurementGate):
        return cirq.PointOptimizationSummary(clear_span=1,
                                             new_operations=[],
                                             clear_qubits=op.qubits)
      else:
        return None

q=cirq.LineQubit(0)
c=cirq.Circuit(cirq.X(q), cirq.measure(q))
print('Before optimization')
print(c)
RemoveMeasurements().optimize_circuit(c)
print('After optimization')
print(c)
